## 1. Preprocessing Dataset

Class imbalance: Check if dataset is balanced; might need weighted loss

Overfitting: Monitor train vs validation gap; apply dropout/augmentation

Computational efficiency: Start with smaller images/models, scale up if needed

Reproducibility: Set random seeds for consistent results


Must Have 

✅ Architecture details (layers, filters, activations)

✅ Final hyperparameters (LR, epochs, batch size, optimizer, loss)

✅ Training/Validation accuracy plot

✅ Test accuracy

✅ Precision, Recall, F1 per class

✅ Confusion matrix

✅ Brief analysis/observations

## 2. CNN architecture

## 3. Training Loop

## 4. Evaluation